# Group Seminar on Julia Pt. II
# Parallel Programming

- Julia's parallel architecture, like MPI, uses message passing between worker processes.
- Distributed Memory architecture $\rightarrow$ Each worker has its own memory.
- It is however more transparent and high level.

## 1. Add worker processes.

In [2]:
nworkers()

1

In [3]:
addprocs(2);nworkers()

2

In [4]:
nprocs()

3

   ## 2. Running operations

In [6]:
r = remotecall(2, +, 1,1)

RemoteRef{Channel{Any}}(2,1,4)

In [7]:
fetch(r)

2

In [10]:
remotecall_fetch(2, +, 1,1)

2

### `remotecall` and `fetch` are pretty low level.
### Julia offers convenient macros `@spawn`, `@spawnat`, `@fetch`

In [11]:
@spawn svd(rand(10,10))

RemoteRef{Channel{Any}}(2,1,7)

In [12]:
fetch(ans)

(
10x10 Array{Float64,2}:
 -0.354644   0.0428627  -0.112984   …  -0.170354    -0.722049    0.34209  
 -0.286629  -0.409146   -0.424579       0.219976    -0.0915964   0.113647 
 -0.312862   0.48877    -0.209538      -0.28515      0.33282    -0.0534861
 -0.264664  -0.487777   -0.0347126     -0.408563    -0.0273398  -0.673535 
 -0.346969  -0.400147    0.500845       0.118572     0.403286    0.413749 
 -0.280033   0.252234   -0.311212   …   0.520383     0.143362   -0.179214 
 -0.302472  -0.123317   -0.360248       0.00978024   0.1793      0.217591 
 -0.330695   0.278419    0.327295      -0.310196    -0.120341    0.0975759
 -0.344226   0.136754    0.420639       0.502632    -0.260286   -0.388254 
 -0.32557    0.137022    0.0178639     -0.201351     0.247319   -0.0513207,

[5.653305195414068,1.434729128855979,1.2544345743643428,1.1569702515069695,0.9297520902119544,0.6045365669031618,0.44331529934692804,0.32876394211731075,0.21037862702841545,0.08416388420730403],
10x10 Array{Float64,2}:
 -0

In [13]:
@fetch svd(rand(10,10))

(
10x10 Array{Float64,2}:
 -0.244836   0.589164   -0.123841  …   0.153932   -0.382087   -0.0878293
 -0.370077  -0.501515   -0.206728      0.0555734  -0.119501    0.0792905
 -0.21358    0.282188    0.623024      0.221814    0.462198    0.358727 
 -0.297126  -0.0550202  -0.157274      0.152709    0.181648   -0.0796965
 -0.316495   0.414319   -0.278304     -0.0594095  -0.238312    0.343191 
 -0.360476  -0.043308   -0.316002  …  -0.171035    0.545451    0.133393 
 -0.316531   0.183103    0.103247     -0.648769    0.156738   -0.529799 
 -0.30925   -0.103983   -0.237181      0.39664     0.0672957  -0.0141099
 -0.350893  -0.119442    0.371367      0.395362   -0.167696   -0.524198 
 -0.346601  -0.294226    0.383268     -0.36521    -0.426121    0.399648 ,

[5.313235205107073,1.4511706988049402,1.268525582449241,1.1640376258988399,0.88220256581468,0.8162893289084378,0.7157037936668676,0.4629337588658518,0.3449206026183042,0.05905873755138128],
10x10 Array{Float64,2}:
 -0.298372   0.429544    0.6